In [1]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.feature_extraction.text import TfidfVectorizer
from langchain.embeddings.base import Embeddings
from ebooklib import epub
from bs4 import BeautifulSoup
import numpy as np


/opt/miniconda3/envs/zyhdl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = QdrantClient(host="localhost", port=6333)

In [3]:
collections = client.get_collections()
print(collections)

collections=[CollectionDescription(name='demo_collection')]


In [8]:
query = "葡萄糖的作用是什么"
hybrid_results = hybrid_search_from_db(query, vector_store, tfidf_embedder)
for result in hybrid_results:
    print("Retrieved:", result)


NameError: name 'vector_store' is not defined

In [1]:


class TfidfSparseEmbeddings(Embeddings):
    def __init__(self):
        self.vectorizer = TfidfVectorizer()

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        self.matrix = self.vectorizer.fit_transform(texts)
        return self.matrix.toarray().tolist()

    def embed_query(self, query: str) -> list[float]:
        q_vector = self.vectorizer.transform([query])
        return q_vector.toarray().tolist()[0]


def split_document(content: str, chunk_size: int = 500, chunk_overlap: int = 50):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(content)


def extract_text_from_epub(epub_path: str) -> str:
    book = epub.read_epub(epub_path)
    content_pieces = []
    for item in book.get_items():
        if item.get_type() == 9:
            soup = BeautifulSoup(item.get_body_content(), "html.parser")
            text = soup.get_text(strip=True)
            if text:
                content_pieces.append(text)
    return "\n".join(content_pieces)


def process_and_store_epub(file_path: str, vector_store, tfidf_embedder, chunk_size: int = 500, chunk_overlap: int = 50):
    content = extract_text_from_epub(file_path)
    chunks = split_document(content, chunk_size, chunk_overlap)
    documents = [Document(page_content=chunk) for chunk in chunks]

    # 向量化存储
    ids = [i for i in range(len(documents))]
    vector_store.add_documents(documents=documents, ids=ids)

    # TF-IDF 向量化
    tfidf_embedder.embed_documents([doc.page_content for doc in documents])
    print(f"Processed and stored {len(documents)} chunks from {file_path}")


def hybrid_search(query: str, vector_store, tfidf_embedder, documents, top_k: int = 5):
    # 向量检索
    retriever = vector_store.as_retriever(search_kwargs={"k": top_k})
    vector_results = retriever.invoke(query)

    # 提取向量检索结果中的内容
    vector_result_contents = [doc.page_content for doc in vector_results]

    # 关键词检索（TF-IDF）
    query_vector = tfidf_embedder.embed_query(query)
    doc_vectors = np.array(tfidf_embedder.matrix.toarray())
    similarities = doc_vectors @ query_vector  # 矩阵乘法计算相似度
    keyword_indices = similarities.argsort()[-top_k:][::-1]
    keyword_results = [documents[i].page_content for i in keyword_indices]

    # 合并结果
    combined_results = list(set(vector_result_contents + keyword_results))

    # 返回对应的 Document 对象
    return [doc for doc in documents if doc.page_content in combined_results]



# 初始化 Qdrant 客户端
client = QdrantClient(host="localhost", port=6333)

# 检查模型生成的密集向量维度
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", cache_folder="../embedding_model")
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    cache_folder="../embedding_model",
)
example_vector = model.encode("Sample text")
vector_size = len(example_vector)

# 创建或重建 Qdrant 集合
client.recreate_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
)

# 初始化向量存储和 TF-IDF Embedding
vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embedding,
    retrieval_mode="dense",
)
tfidf_embedder = TfidfSparseEmbeddings()

# 示例：处理 EPUB 文件
epub_file_path = "/Users/zhaoyuhang/Downloads/生酮饮食：低碳水、高脂肪饮食完全指南 (吉米·摩尔（Jimmy Moore）  埃里克·韦斯特曼（Eric Westman）) (Z-Library).epub"
process_and_store_epub(epub_file_path, vector_store, tfidf_embedder)




/opt/miniconda3/envs/zyhdl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/_n/9zf09hdj0fz1tnnwcsxd7gs00000gn/T/ipykernel_14944/1772252649.py:87: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/var/folders/_n/9zf09hdj0fz1tnnwcsxd7gs00000gn/T/ipykernel_14944/1772252649.py:95: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence a

Processed and stored 483 chunks from /Users/zhaoyuhang/Downloads/生酮饮食：低碳水、高脂肪饮食完全指南 (吉米·摩尔（Jimmy Moore）  埃里克·韦斯特曼（Eric Westman）) (Z-Library).epub
Retrieved: 生酮饮食专家介绍通过我的播客，我有幸采访到上百个最优秀、最聪明的专家，谈论了许多健康方面的重要话题。因此，决定写这本书的时候，我就知道该去找谁询问关于生酮饮食以及它对健康影响的相关信息。我很高兴能介绍这22位来自世界各地的专家。你会在本书中读到很多引自他们的话，这些话出现在下文“专家解析”栏目中。兹沙恩·阿伦（Zeeshan
Retrieved: Arain），医学学士、公共卫生与热带医学硕士、澳大利亚皇家全科医学院院士阿伦博士在澳大利亚墨尔本的莫纳什大学拿到了他的医学学位，他也拿到了詹姆斯·库克大学的公共卫生与热带医学学士学位。阿伦博士作为一名全科医生，在澳大利亚足球联盟（AFL）墨尔本足球俱乐部任职，这是全世界最精英的专业体育组织之一。阿伦博士对营养和运动在预防和治疗慢性疾病和肥胖中的作用特别感兴趣。阿伦博士亲自治疗了数百名患者，他使用配制好的低碳水、高脂肪的生酮饮食来控制各种医疗状况，包括糖尿病、多囊卵巢综合征、肥胖症、高血压、癫痫、胃食管反流和肠易激综合征。他已经就生酮饮食进行了几次公开讲座，也正在开展这一领域的研究。个人生活中，他自2012年以来一直在进行生酮饮食。请在网站SouthYarraMedical.com.au/doctors/5/dr-zeeshan-arain上了解更多关于阿伦博士的信息。布莱恩·巴克斯代尔（Bryan
Retrieved: Runyan），医学博士鲁尼恩博士是佛罗里达州圣彼得斯堡私人诊所的执业医师，专攻内科、肾脏病和肥胖医学。他在2001年开始私人执业之前，曾经做过10年的急救医疗。1998年，他在38岁时患上了1型糖尿病。尽管他的糖尿病在强化胰岛素治疗下得到了很好的控制，但他经常发生低血糖。2011年，在铁人三项赛训练时，鲁尼恩博士开始寻找一种更好地治疗糖尿病并进行耐力运动的方法，他决定尝试使用低碳水、高脂肪的生酮饮食。2012年2月，他开始用这种饮食来治疗自己的糖尿病，并得知这种饮食对于治疗许多其他疾病（包括肥胖

In [3]:
# 查询示例
documents = [Document(page_content=chunk) for chunk in split_document(extract_text_from_epub(epub_file_path))]
query = "葡萄糖的作用是什么"
hybrid_results = hybrid_search(query, vector_store, tfidf_embedder, documents)
for result in hybrid_results:
    print("Retrieved:", result.page_content)

/opt/miniconda3/envs/zyhdl/lib/python3.12/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/opt/miniconda3/envs/zyhdl/lib/python3.12/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


Retrieved: A没有对任何人提到过关于生酮饮食在控制血糖和改善健康方面的治疗作用——包括对数百万能受益于生酮饮食的2型糖尿病患者们（我们会在第16章详细讨论）。这简直是个悲剧！鉴于ADA的读者都是糖尿病患者，这些人酮症酸中毒的风险最高，所以ADA的担心是可以理解的。但是只要这些患者的血糖保持在较低状态，他们其实并不需要害怕生酮——既然研究已表明生酮对控制糖尿病可能是非常有益的（我们会在第16章详细讨论），糖尿病患者也许还会享受到很多好处呢！资料来源：Diabetes.org.专家解析我们的研究实验表明，生酮状态最大的益处就是葡萄糖代谢显著降低。这与糖尿病症状正好相反。——查尔斯·莫伯斯博士正如这些受到高度关注的、所谓的健康权威组织曾经团结一致地将胆固醇指控为心脏病的罪魁祸首（详见我的另一本书《胆固醇入门全书》）一样，它们也联合起来将生酮状态描述为不良和危险。这些来自医生、营养师以及什么都知道的大师的信息其实都是假的。这也是为什么我们觉得要写下本书：我们要把真正的生酮展示给大家，这些真相几乎和那些健康组织说的完全相反。已有研究显示生酮饮食具有某些治疗效果，这让很多医生和其他健康从业者开始认识到它，发现
Retrieved: 们可能破坏你对食物的消化和代谢。”专家解析当谈到碳水化合物和简单糖时，可以遵循我的简单规则，当你对某种食物有疑问的时候，扔掉它！碳水化合物应该主要来自非淀粉类蔬菜，而不是淀粉和精制碳水化合物。对于有任何程度胰岛素抵抗的人群（如今几乎人人都有），如果你每日碳水摄入大于50克，那么实现并保持生酮的可能性很小。而对于那些没有胰岛素抵抗的人，多摄入一点碳水也许还可以保持生酮状态。——威廉·威尔逊博士两周过后，如果你正在产酮（我们会在第8章中详细介绍如何测量酮体），请尝试慢慢地将每日碳水化合物提高5～10克，持续1周，看看是否会影响你的产酮。如果酮体水平仍然保持在一个合理的营养性生酮状态，那么代表你的饮食中可以多一点碳水化合物。继续重复试探几周，直到你的酮体水平开始下降，这时候恢复到上一个碳水摄入值，保持足够的酮体水平就可以了。如果你每天只摄入20克总碳水，两周后还没有产酮，那么考虑将碳水摄入量再调低至每天10～15克，并且限制蛋白质的摄入（我们将在下一章详细讨论这个话题）。不要失去希望。即使你对碳水化合物特别敏感，也可以进入生酮状态。相信我，我完全